In [120]:
import pandas as pd
import duckdb

from sklearn import datasets
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
import pickle
import joblib

from sklearn import linear_model
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [1]:
df = pd.read_csv('../data/diamonds_test.csv')
df_2 = pd.read_csv('../data/sample_submission.csv')

NameError: name 'pd' is not defined

In [77]:
df

,id,carat,cut,color,clarity,depth,table,x,y,z,city
0,0,0.79,Very Good,F,SI1,62.7,60.0,5.82,5.89,3.67,Amsterdam
1,1,1.20,Ideal,J,VS1,61.0,57.0,6.81,6.89,4.18,Surat
2,2,1.57,Premium,H,SI1,62.2,61.0,7.38,7.32,4.57,Kimberly
3,3,0.90,Very Good,F,SI1,63.8,54.0,6.09,6.13,3.90,Kimberly
4,4,0.50,Very Good,F,VS1,62.9,58.0,5.05,5.09,3.19,Amsterdam
...,...,...,...,...,...,...,...,...,...,...,...
13480,13480,0.57,Ideal,E,SI1,61.9,56.0,5.35,5.32,3.30,Amsterdam
13481,13481,0.71,Ideal,I,VS2,62.2,55.0,5.71,5.73,3.56,New York City
13482,13482,0.70,Ideal,F,VS1,61.6,55.0,5.75,5.71,3.53,Tel Aviv
13483,13483,0.70,Very Good,F,SI2,58.8,57.0,5.85,5.89,3.45,Surat


In [5]:
df_2

,id,price
0,0,685.639121
1,1,642.445785
2,2,746.907960
3,3,60.737590
4,4,650.451526
...,...,...
13480,13480,953.289190
13481,13481,861.357109
13482,13482,985.095652
13483,13483,907.141104


In [6]:
con = duckdb.connect('../data/diamonds_train.db')
df = con.execute('show tables').fetch_df()

In [7]:
df_diamonds = con.execute(
'''SELECT dt.index_id, dt.carat, dcut.cut, dcolor.color, dcl.clarity, dd.depth, dd.table, dd.x, dd.y, dd.z, dc.city, dt.price 
FROM diamonds_transactional AS dt
INNER JOIN diamonds_city AS dc ON dt.city_id = dc.city_id
INNER JOIN diamonds_properties AS dp ON dt.index_id = dp.index_id
INNER JOIN diamonds_clarity AS dcl ON dp.clarity_id = dcl.clarity_id
INNER JOIN diamonds_color AS dcolor ON dp.color_id = dcolor.color_id
INNER JOIN diamonds_cut AS dcut ON dp.cut_id = dcut.cut_id
INNER JOIN diamonds_dimensions AS dd ON dt.index_id = dd.index_id
''').fetch_df()

In [8]:
df_diamonds.columns

Index(['index_id', 'carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x',
       'y', 'z', 'city', 'price'],
      dtype='object')

In [18]:
df_diamonds.drop(columns='index_id', inplace=True)
df_diamonds

,carat,cut,color,clarity,depth,table,x,y,z,city,price
0,1.21,Premium,J,VS2,62.4,58.0,6.83,6.79,4.25,Dubai,4268
1,0.32,Very Good,H,VS2,63.0,57.0,4.35,4.38,2.75,Kimberly,505
2,0.71,Fair,G,VS1,65.5,55.0,5.62,5.53,3.65,Las Vegas,2686
3,0.41,Good,D,SI1,63.8,56.0,4.68,4.72,3.00,Kimberly,738
4,1.02,Ideal,G,SI1,60.5,59.0,6.55,6.51,3.95,Dubai,4882
...,...,...,...,...,...,...,...,...,...,...,...
40450,1.08,Ideal,H,VS2,61.9,54.0,6.64,6.61,4.10,Antwerp,5850
40451,1.15,Ideal,I,SI1,61.8,55.0,6.73,6.76,4.17,Luxembourg,6300
40452,0.53,Ideal,F,VS1,61.4,57.0,5.18,5.20,3.19,Kimberly,1800
40453,0.66,Premium,F,VS1,60.8,58.0,5.67,5.64,3.44,Amsterdam,2368


In [20]:
cols = df_diamonds.columns

In [26]:
cat_list = []
for col in cols:
    cat = df_diamonds[col].unique()
    cat_num = len(cat)
    cat_dict = {"categorical_variable":col,
                "number_of_possible_values":cat_num,
                "values":cat}
    cat_list.append(cat_dict)
categ = pd.DataFrame(cat_list).sort_values(by="number_of_possible_values")

In [27]:
categ

,categorical_variable,number_of_possible_values,values
1,cut,5,"[Premium, Very Good, Fair, Good, Ideal]"
2,color,7,"[J, H, G, D, F, E, I]"
3,clarity,8,"[VS2, VS1, SI1, SI2, IF, VVS1, VVS2, I1]"
9,city,13,"[Dubai, Kimberly, Las Vegas, Tel Aviv, Zurich,..."
5,table,121,"[58.0, 57.0, 55.0, 56.0, 59.0, 54.2, 63.0, 61...."
4,depth,175,"[62.4, 63.0, 65.5, 63.8, 60.5, 61.2, 62.3, 61...."
0,carat,270,"[1.21, 0.32, 0.71, 0.41, 1.02, 1.52, 0.31, 1.1..."
8,z,363,"[4.25, 2.75, 3.65, 3.0, 3.95, 4.54, 4.57, 2.69..."
7,y,539,"[6.79, 4.38, 5.53, 4.72, 6.51, 7.39, 7.35, 4.3..."
6,x,543,"[6.83, 4.35, 5.62, 4.68, 6.55, 7.45, 7.32, 4.3..."


In [32]:
ohe = pd.get_dummies(df_diamonds, drop_first=True)

In [33]:
ohe.columns

Index(['carat', 'depth', 'table', 'x', 'y', 'z', 'price', 'cut_Good',
       'cut_Ideal', 'cut_Premium', 'cut_Very Good', 'color_E', 'color_F',
       'color_G', 'color_H', 'color_I', 'color_J', 'clarity_IF', 'clarity_SI1',
       'clarity_SI2', 'clarity_VS1', 'clarity_VS2', 'clarity_VVS1',
       'clarity_VVS2', 'city_Antwerp', 'city_Dubai', 'city_Kimberly',
       'city_Las Vegas', 'city_London', 'city_Luxembourg', 'city_Madrid',
       'city_New York City', 'city_Paris', 'city_Surat', 'city_Tel Aviv',
       'city_Zurich'],
      dtype='object')

In [34]:
ohe

,carat,depth,table,x,y,z,price,cut_Good,cut_Ideal,cut_Premium,...,city_Kimberly,city_Las Vegas,city_London,city_Luxembourg,city_Madrid,city_New York City,city_Paris,city_Surat,city_Tel Aviv,city_Zurich
0,1.21,62.4,58.0,6.83,6.79,4.25,4268,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0.32,63.0,57.0,4.35,4.38,2.75,505,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,0.71,65.5,55.0,5.62,5.53,3.65,2686,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,0.41,63.8,56.0,4.68,4.72,3.00,738,1,0,0,...,1,0,0,0,0,0,0,0,0,0
4,1.02,60.5,59.0,6.55,6.51,3.95,4882,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40450,1.08,61.9,54.0,6.64,6.61,4.10,5850,0,1,0,...,0,0,0,0,0,0,0,0,0,0
40451,1.15,61.8,55.0,6.73,6.76,4.17,6300,0,1,0,...,0,0,0,1,0,0,0,0,0,0
40452,0.53,61.4,57.0,5.18,5.20,3.19,1800,0,1,0,...,1,0,0,0,0,0,0,0,0,0
40453,0.66,60.8,58.0,5.67,5.64,3.44,2368,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [35]:
feat = ['carat', 'depth', 'table', 'x', 'y', 'z', 'cut_Good',
       'cut_Ideal', 'cut_Premium', 'cut_Very Good', 'color_E', 'color_F',
       'color_G', 'color_H', 'color_I', 'color_J', 'clarity_IF', 'clarity_SI1',
       'clarity_SI2', 'clarity_VS1', 'clarity_VS2', 'clarity_VVS1',
       'clarity_VVS2', 'city_Antwerp', 'city_Dubai', 'city_Kimberly',
       'city_Las Vegas', 'city_London', 'city_Luxembourg', 'city_Madrid',
       'city_New York City', 'city_Paris', 'city_Surat', 'city_Tel Aviv',
       'city_Zurich']
target = ['price']

In [36]:
X, y = ohe[feat], ohe[target]

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [44]:
model = LinearRegression()
hyperparameters = model.get_params()
print(f'hyperparameters: {hyperparameters}',
      f'\nmodel: {model}')

hyperparameters: {'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'positive': False} 
model: LinearRegression()


In [46]:
model.fit(X_train, y_train)

LinearRegression()

In [47]:
predictions = model.predict(X_test)

In [122]:
rmse = mean_squared_error(y_test, predictions)**0.5
rmse

1137.6633380989135

In [58]:
y_test.shape

(8091, 1)

In [64]:
p = pd.DataFrame(predictions, columns=['prediction'])
p

,prediction
0,9598.563035
1,13.455362
2,2171.619023
3,131.710883
4,1459.858376
...,...
8086,1296.453211
8087,2001.127112
8088,5577.817462
8089,7166.625893


In [73]:
check = y_test.reset_index().drop(columns='index').join(p)
check['dif'] = check['price'] - check['prediction']

In [74]:
check

,price,prediction,dif
0,7486,9598.563035,-2112.563035
1,571,13.455362,557.544638
2,1852,2171.619023,-319.619023
3,544,131.710883,412.289117
4,1284,1459.858376,-175.858376
...,...,...,...
8086,814,1296.453211,-482.453211
8087,1183,2001.127112,-818.127112
8088,5312,5577.817462,-265.817462
8089,4520,7166.625893,-2646.625893


In [78]:
df

,id,carat,cut,color,clarity,depth,table,x,y,z,city
0,0,0.79,Very Good,F,SI1,62.7,60.0,5.82,5.89,3.67,Amsterdam
1,1,1.20,Ideal,J,VS1,61.0,57.0,6.81,6.89,4.18,Surat
2,2,1.57,Premium,H,SI1,62.2,61.0,7.38,7.32,4.57,Kimberly
3,3,0.90,Very Good,F,SI1,63.8,54.0,6.09,6.13,3.90,Kimberly
4,4,0.50,Very Good,F,VS1,62.9,58.0,5.05,5.09,3.19,Amsterdam
...,...,...,...,...,...,...,...,...,...,...,...
13480,13480,0.57,Ideal,E,SI1,61.9,56.0,5.35,5.32,3.30,Amsterdam
13481,13481,0.71,Ideal,I,VS2,62.2,55.0,5.71,5.73,3.56,New York City
13482,13482,0.70,Ideal,F,VS1,61.6,55.0,5.75,5.71,3.53,Tel Aviv
13483,13483,0.70,Very Good,F,SI2,58.8,57.0,5.85,5.89,3.45,Surat


In [79]:
ohe_2 = pd.get_dummies(df, drop_first=True)
ohe_2

,id,carat,depth,table,x,y,z,cut_Good,cut_Ideal,cut_Premium,...,city_Kimberly,city_Las Vegas,city_London,city_Luxembourg,city_Madrid,city_New York City,city_Paris,city_Surat,city_Tel Aviv,city_Zurich
0,0,0.79,62.7,60.0,5.82,5.89,3.67,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1.20,61.0,57.0,6.81,6.89,4.18,0,1,0,...,0,0,0,0,0,0,0,1,0,0
2,2,1.57,62.2,61.0,7.38,7.32,4.57,0,0,1,...,1,0,0,0,0,0,0,0,0,0
3,3,0.90,63.8,54.0,6.09,6.13,3.90,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,4,0.50,62.9,58.0,5.05,5.09,3.19,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13480,13480,0.57,61.9,56.0,5.35,5.32,3.30,0,1,0,...,0,0,0,0,0,0,0,0,0,0
13481,13481,0.71,62.2,55.0,5.71,5.73,3.56,0,1,0,...,0,0,0,0,0,1,0,0,0,0
13482,13482,0.70,61.6,55.0,5.75,5.71,3.53,0,1,0,...,0,0,0,0,0,0,0,0,1,0
13483,13483,0.70,58.8,57.0,5.85,5.89,3.45,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [81]:
ohe_2.drop(columns='id', inplace=True)
ohe_2

,carat,depth,table,x,y,z,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,...,city_Kimberly,city_Las Vegas,city_London,city_Luxembourg,city_Madrid,city_New York City,city_Paris,city_Surat,city_Tel Aviv,city_Zurich
0,0.79,62.7,60.0,5.82,5.89,3.67,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1.20,61.0,57.0,6.81,6.89,4.18,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,1.57,62.2,61.0,7.38,7.32,4.57,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
3,0.90,63.8,54.0,6.09,6.13,3.90,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
4,0.50,62.9,58.0,5.05,5.09,3.19,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13480,0.57,61.9,56.0,5.35,5.32,3.30,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
13481,0.71,62.2,55.0,5.71,5.73,3.56,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
13482,0.70,61.6,55.0,5.75,5.71,3.53,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
13483,0.70,58.8,57.0,5.85,5.89,3.45,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [82]:
pred = model.predict(ohe_2)

In [103]:
p_df = pd.DataFrame(pred, columns=['Price'])

In [105]:
p_df.reset_index(inplace=True)

In [111]:
p_df.rename(columns={"index": "id"}, inplace=True)

In [116]:
p_df = p_df.set_index('id')

In [118]:
p_df.to_csv('../data/test.csv')

In [117]:
p_df

,Price
id,
0,3666.823816
1,6321.153505
2,10142.059471
3,4653.276260
4,2150.241169
...,...
13480,2002.909129
13481,2499.679731
13482,3891.492542
